In [1]:
from PIL import Image
import numpy as np
import os
import tensorflow as tf
from tensorflow.python.keras.layers.convolutional import Conv2D
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers.convolutional import MaxPooling2D
from tensorflow.python.keras.layers import Flatten
from sklearn.model_selection import train_test_split

!git clone https://github.com/jlim10343/skin_images
os.chdir('./skin_images')

Cloning into 'skin_images'...
remote: Enumerating objects: 4664, done.
remote: Counting objects: 100% (4664/4664), done.
remote: Compressing objects: 100% (4662/4662), done.
remote: Total 4664 (delta 0), reused 4661 (delta 0), pack-reused 0
Receiving objects: 100% (4664/4664), 35.27 MiB | 9.72 MiB/s, done.


In [0]:
eczema_images = np.array([np.array(Image.open('Eczema/' + filename)) for filename in os.listdir('Eczema')])
tinea_images = np.array([np.array(Image.open('Tinea/' + filename)) for filename in os.listdir('Tinea')])
psoriasis_images = np.array([np.array(Image.open('Psoriasis/' + filename)) for filename in os.listdir('Psoriasis')])
skin_images = np.array([np.array(Image.open('Skin_pics/' + filename)) for filename in os.listdir('Skin_pics')])

In [0]:
eczema_images = np.reshape(eczema_images, (eczema_images.shape[0], 224, 224, 3))
tinea_images = np.reshape(tinea_images, (tinea_images.shape[0], 224, 224, 3))
psoriasis_images = np.reshape(psoriasis_images, (psoriasis_images.shape[0], 224, 224, 3))
skin_images = np.reshape(skin_images, (skin_images.shape[0], 224, 224, 3))

In [0]:
tinea_images = tinea_images.astype(np.float32) / 255
eczema_images = eczema_images.astype(np.float32) / 255
psoriasis_images = psoriasis_images.astype(np.float32) / 255
skin_images = skin_images.astype(np.float32) / 255

In [0]:
images = np.vstack((eczema_images, psoriasis_images, tinea_images, skin_images))

In [0]:
# [1, 0, 0, 0] is eczema, [0, 1, 0, 0] is psoriasis, [0, 0, 1, 0] is tinea, and [0, 0, 0, 1] is normal skin
labels = [[1, 0, 0, 0]] * eczema_images.shape[0] + [[0, 1, 0, 0]] * psoriasis_images.shape[0] + [[0, 0, 1, 0]] * tinea_images.shape[0] + [[0, 0, 0, 1]] * skin_images.shape[0]
labels = np.array(labels)

In [0]:
model = tf.keras.Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(4, activation='softmax'))
# We will now compile and print out a summary of our model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
data_train, data_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.20, random_state=0)

In [0]:
model.fit(data_train, labels_train, epochs=5)

Epoch 1/5
3857/3857 [==============================] - 377s 98ms/sample - loss: 1.9962 - acc: 0.3938
Epoch 2/5
3857/3857 [==============================] - 374s 97ms/sample - loss: 1.1282 - acc: 0.4952
Epoch 3/5
3857/3857 [==============================] - 373s 97ms/sample - loss: 1.0455 - acc: 0.5380
Epoch 4/5
  32/3857 [..............................] - ETA: 6:09 - loss: 0.7205 - acc: 0.7812

In [0]:
loss, accuracy = model.evaluate(data_test, labels_test)
print('Test accuracy: %.2f' % (accuracy))